# atmodeller

## Tutorial 4: Interior (include Si in the atmosphere)

Import the required functionality and activate the debug logger.

In [ ]:
from atmodeller import debug_logger, debug_file_logger
from atmodeller.constraints import  MassConstraint, SystemConstraints, IronWustiteBufferConstraintHirschmann, FugacityConstraint
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Planet, Species
from atmodeller.interfaces import GasSpecies, LiquidSpecies
from atmodeller.solubilities import PeridotiteH2O, BasaltDixonCO2
from atmodeller.utilities import earth_oceans_to_kg

# debug_file_logger()
debug_logger()

### 1. H2O-H2-O2-SiO-SiH4-SiO2l system

We define a list of the species we wish to include in the interior-atmosphere system

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))#, solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))


Create an interior-atmosphere system using H-O-Si for a sub-Neptune (Planet mass = 11 M_Earth, Interior radius = 1.7 R_Earth)

In [ ]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

To solve the system, we provide a constraint of the H2O and O2 fugacity in bar and assume the actvity of SiO2(l)=1.

In [ ]:
constraints: SystemConstraints = SystemConstraints(
    [
        FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of approx 10 kbar (1 GPa)
        FugacityConstraint(species='O2Si', value=1),  # This constraints simulates activity of 1
        IronWustiteBufferConstraintHirschmann(log10_shift=-4), # This constraint sets a low fO2 simulating a sub-Neptune surface
    ])


In [ ]:
interior_atmosphere.solve(constraints)

You can access the solution directly using:

In [ ]:
interior_atmosphere.solution_dict

You can access more information about the solution using:

In [ ]:
interior_atmosphere.output

### 2. H2O-H2-O2-SiO-SiH4-SiO2l-CH4-CO-CO2 system

We now extend the species list to additionally include C-species:

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CH4'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))

species

In [ ]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

An additional constraint is required to solve the system

In [ ]:
constraints: SystemConstraints = SystemConstraints(
    [
        FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of 10 kbar (1 GPa)
        FugacityConstraint(species='O2Si', value=1),  # This constraints simulates activity of 1
        FugacityConstraint(species='CO', value=1),  # This is an arbitrary constraint
        IronWustiteBufferConstraintHirschmann(log10_shift=-4),  # This constraint sets a low fO2 simulating a sub-Neptune surface
    ])


In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict

### 3. System with mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of fugacity and mass constraints. We define the same species as before:

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CH4'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))

species

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [ ]:
number_of_earth_oceans: float = 11 # 11x to account for mass increase by 11x
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans) # note: this constraint is not used below
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of 10 kbar (1 GPa)
    # MassConstraint(species="H", value=mass_H), # Mass constraint of H
    MassConstraint(species="C", value=mass_C), # Mass constraint of C
    FugacityConstraint(species='O2Si', value=1), # Simulates an activity of unity
    IronWustiteBufferConstraintHirschmann(log10_shift=-4) # Sets a low fO2 for a sub-Neptune surface
])

In [ ]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

Solution

In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict

### 4. System with mixed constraints and H2O-CO2 solubilities

Include H2O and CO2 solubilities

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CH4'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))

species

In [ ]:
number_of_earth_oceans: float = 11 # 11x to account for mass increase by 11x
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans) # note: this constraint is not used below
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of 10 kbar (1 GPa)
    # MassConstraint(species="H", value=mass_H), # Mass constraint of H
    MassConstraint(species="C", value=mass_C), # Mass constraint of C
    FugacityConstraint(species='O2Si', value=1), # Simulates an activity of unity
    IronWustiteBufferConstraintHirschmann(log10_shift=-4) # Sets a low fO2 for a sub-Neptune surface
])

In [ ]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict